In [1]:
import pandas as pd
import numpy as np
import os
import rasterio 
from rasterio.transform import xy
import geopandas as gpd
from rasterio.features import geometry_mask
from shapely.geometry import Point

In [2]:

bil_file = r'C:\Users\yegor\Desktop\Honors Thesis\Data\Temperature\PRISM_tmean_stable_4kmD2_20190101_20191231_bil\PRISM_tmean_stable_4kmD2_20190101_bil.bil'

with rasterio.open(bil_file) as dataset:
    tmean_data = dataset.read(1)  # temp data
    affine = dataset.transform  # transformation matrix
    crs = dataset.crs  # coordinates
    bounds = dataset.bounds  # geo bounds
    
    print(f"Temperature Data Shape: {tmean_data.shape}")
    print(f"Affine Transformation: {affine}")
    print(f"Coordinate Reference System: {crs}")
    print(f"Geographic Bounds: {bounds}")
    
    print("First 5 rows and columns of temperature data:")
    print(tmean_data[:5, :5])  #top left 5x5 tempt data


Temperature Data Shape: (621, 1405)
Affine Transformation: | 0.04, 0.00,-125.02|
| 0.00,-0.04, 49.94|
| 0.00, 0.00, 1.00|
Coordinate Reference System: OGC:CRS83
Geographic Bounds: BoundingBox(left=-125.0208333333335, bottom=24.0624999997925, right=-66.4791666661985, top=49.9374999999995)
First 5 rows and columns of temperature data:
[[-9999. -9999. -9999. -9999. -9999.]
 [-9999. -9999. -9999. -9999. -9999.]
 [-9999. -9999. -9999. -9999. -9999.]
 [-9999. -9999. -9999. -9999. -9999.]
 [-9999. -9999. -9999. -9999. -9999.]]


In [3]:
bil_file = r'C:\Users\yegor\Desktop\Honors Thesis\Data\Temperature\PRISM_tmean_stable_4kmD2_20190101_20191231_bil\PRISM_tmean_stable_4kmD2_20190101_bil.bil'
with rasterio.open(bil_file) as dataset:
    tmean_data = dataset.read(1)  #first band is temp data
    affine = dataset.transform  # transformation matric
    crs = dataset.crs  # coordinate refernce system

    valid_temperature_data = np.ma.masked_equal(tmean_data, -9999) # mask "missing values"
    
    # row col of missing
    valid_indices = np.argwhere(valid_temperature_data.mask == False)

    # conversion to coordinates
    row, col = valid_indices[0]
    lon, lat = rasterio.transform.xy(affine, row, col, offset='center')
    
    print(f"First valid temperature value: {valid_temperature_data[row, col]}")
    print(f"Corresponding coordinates: Longitude = {lon}, Latitude = {lat}")


First valid temperature value: -24.99700164794922
Corresponding coordinates: Longitude = -95.124999999761, Latitude = 49.416666666662


In [4]:
bil_file = r'C:\Users\yegor\Desktop\Honors Thesis\Data\Temperature\PRISM_tmean_stable_4kmD2_20190101_20191231_bil\PRISM_tmean_stable_4kmD2_20190402_bil.bil'
with rasterio.open(bil_file) as dataset:
    tmean_data = dataset.read(1)  #first band is temp data
    affine = dataset.transform  # transformation matric
    crs = dataset.crs  # coordinate refernce system

    valid_temperature_data = np.ma.masked_equal(tmean_data, -9999) # mask "missing values"
    
    # row col of missing
    valid_indices = np.argwhere(valid_temperature_data.mask == False)

    # conversion to coordinates
    row, col = valid_indices[0]
    lon, lat = rasterio.transform.xy(affine, row, col, offset='center')
    
    print(f"First valid temperature value: {valid_temperature_data[row, col]}")
    print(f"Corresponding coordinates: Longitude = {lon}, Latitude = {lat}")

First valid temperature value: 0.0024999999441206455
Corresponding coordinates: Longitude = -95.124999999761, Latitude = 49.416666666662


In [5]:
temperature_records = []

# Read the .bil file (example: January 1, 2019)
bil_file = r'C:\Users\yegor\Desktop\Honors Thesis\Data\Temperature\PRISM_tmean_stable_4kmD2_20190101_20191231_bil\PRISM_tmean_stable_4kmD2_20190101_bil.bil'

with rasterio.open(bil_file) as dataset:
    tmean_data = dataset.read(1)  
    affine = dataset.transform  

    #mask data
    tmean_data = np.ma.masked_equal(tmean_data, -9999)
    
    #loop through the grid and store valid data
    for row in range(tmean_data.shape[0]):
        for col in range(tmean_data.shape[1]):
            if not tmean_data.mask[row, col]:  #valid data only
                lon, lat = xy(affine, row, col)  #geo coordinate conversion
                temperature = tmean_data[row, col]
                #store as dictionary 
                temperature_records.append({
                    'latitude': lat,
                    'longitude': lon,
                    'temperature': temperature,
                    'date': '2019-01-01'  
                })


df = pd.DataFrame(temperature_records)

In [6]:
df

,latitude,longitude,temperature,date
0,49.416667,-95.125000,-24.997002,2019-01-01
1,49.375000,-95.166667,-24.939001,2019-01-01
2,49.375000,-95.125000,-24.968000,2019-01-01
3,49.375000,-95.083333,-25.041000,2019-01-01
4,49.375000,-95.041667,-25.163002,2019-01-01
...,...,...,...,...
481626,24.541667,-81.666667,25.906002,2019-01-01
481627,24.541667,-81.625000,25.937002,2019-01-01
481628,24.500000,-82.166667,26.131001,2019-01-01
481629,24.500000,-82.125000,26.119001,2019-01-01


**Takes about 15-20 seconds to run one day into full output, need to aggregate by county first. County lines can change slightly over time, multiple solutions (depends on how mortality data would be defined). for instance, if they just give address for mortality, then can just aggregate to modern county lines.**



In [ ]:


counties = gpd.read_file(county_shapefile)                                                                                                    

print(counties.columns) 

counties = counties.to_crs('OGC:CRS83') # would have to loop this in for given FILE
for row in range(valid_temperature_data.shape[0]):
            for col in range(valid_temperature_data.shape[1]):
                if not valid_temperature_data.mask[row, col]:  # only process valid 
                    lon, lat = rasterio.transform.xy(affine, row, col)  #convert to coords
                    temperature = valid_temperature_data[row, col]

                    # point for gril cell
                    point = Point(lon, lat)

                    #find which county this point falls into using spatial join
                    for county in counties.itertuples():
                        if county.geometry.contains(point):
                            #append temperature and county FIPS code
                            county_temperatures.append((county.GEOID, temperature))

In [ ]:
counties = gpd.read_file(r"C:\Users\yegor\Desktop\Honors Thesis\Data\Counties_Boundaries\US_AtlasHCB_Counties\US_HistCounties_Shapefile\US_HistCounties.shp")                                                                                                    


In [10]:
print(counties.columns)

Index(['ID_NUM', 'NAME', 'ID', 'STATE_TERR', 'FIPS', 'VERSION', 'START_DATE',
       'END_DATE', 'CHANGE', 'CITATION', 'START_N', 'END_N', 'AREA_SQMI',
       'CNTY_TYPE', 'FULL_NAME', 'CROSS_REF', 'NAME_START', 'geometry'],
      dtype='object')


In [18]:
counties['START_DATE'] = pd.to_datetime(counties['START_DATE'], errors='coerce')
counties['END_DATE'] = pd.to_datetime(counties['END_DATE'], errors='coerce')

start_date = pd.to_datetime('1981-01-01')
end_date = pd.to_datetime('2000-12-31')

counties = counties[(counties['START_DATE'] <= end_date) & (counties['END_DATE'] >= start_date)]


In [21]:
first_bil_file= r"C:\Users\yegor\Desktop\Honors Thesis\Data\Temperature\PRISM_tmean_stable_4kmD2_19810101_19811231_bil\PRISM_tmean_stable_4kmD2_19810101_bil.bil"
with rasterio.open(first_bil_file) as dataset:
    tmean_data = dataset.read(1)
    affine = dataset.transform
    crs = dataset.crs

    counties = counties.to_crs(crs)
    
    tmean_data = np.ma.masked_equal(tmean_data, -9999)



In [ ]:
date_1981 = pd.to_datetime('1981-01-01')

counties_1981 = counties[(counties['START_DATE'] <= date_1981) & 
                         ((counties['END_DATE'].isna()) | (counties['END_DATE'] >= date_1981))]

print(counties_1981[['FIPS', 'NAME', 'START_DATE', 'END_DATE']].head())

     FIPS                 NAME START_DATE   END_DATE
2   02010     ALEUTIAN ISLANDS 1980-04-01 1990-03-31
9   02020            ANCHORAGE 1980-04-01 1990-03-31
20  02050               BETHEL 1980-04-01 1990-03-31
26  02060  BRISTOL BAY BOROUGH 1980-04-01 1990-03-31
32  02070           DILLINGHAM 1980-04-01 1990-03-31


In [ ]:
counties['START_DATE'] = pd.to_datetime(counties['START_DATE'], errors='coerce')
counties['END_DATE'] = pd.to_datetime(counties['END_DATE'], errors='coerce')

start_date = pd.to_datetime('1981-01-01')
end_date = pd.to_datetime('2000-12-31')

counties = counties[(counties['START_DATE'] <= end_date) & (counties['END_DATE'] >= start_date)]


In [28]:
counties_1981 = counties_1981.to_crs(crs)
points = []
temperatures = []
#step 1: Loop through each valid grid cell to get coordinates and temperature values
for row in range(tmean_data.shape[0]):
    for col in range(tmean_data.shape[1]):
        if not tmean_data.mask[row, col]: 
            lon, lat = rasterio.transform.xy(affine, row, col) 
            temperature = tmean_data[row, col]
            
            points.append(Point(lon, lat))
            temperatures.append(temperature)

In [31]:
points_gdf = gpd.GeoDataFrame({'temperature': temperatures, 'geometry': points}, crs=crs)

points_with_counties = gpd.sjoin(points_gdf, counties_1981[['FIPS', 'geometry']], how='left', predicate='within')


In [40]:
points_with_counties

,temperature,geometry,index_right,FIPS
0,-9.248000,POINT (-95.125 49.41667),NaN,NaN
1,-9.143001,POINT (-95.16667 49.375),NaN,NaN
2,-9.179001,POINT (-95.125 49.375),NaN,NaN
3,-9.204000,POINT (-95.08333 49.375),NaN,NaN
4,-9.052000,POINT (-95.04167 49.375),NaN,NaN
...,...,...,...,...
481626,15.522000,POINT (-81.66667 24.54167),NaN,NaN
481627,15.529000,POINT (-81.625 24.54167),NaN,NaN
481628,16.225000,POINT (-82.16667 24.5),NaN,NaN
481629,16.247002,POINT (-82.125 24.5),NaN,NaN


In [39]:
points_with_counties.size

1926524

In [36]:
filtered_points_with_counties = points_with_counties.dropna(subset=['FIPS'])


In [41]:
filtered_points_with_counties

,temperature,geometry,index_right,FIPS
11,-9.049001,POINT (-95.125 49.33333),8073.0,27077
12,-9.010000,POINT (-95.08333 49.33333),8073.0,27077
13,-9.029000,POINT (-95.04167 49.33333),8073.0,27077
14,-9.027000,POINT (-95 49.33333),8073.0,27077
15,-8.969001,POINT (-94.95833 49.33333),8073.0,27077
...,...,...,...,...
481574,15.556001,POINT (-81.54167 24.66667),2953.0,12087
481575,15.526001,POINT (-81.5 24.66667),2953.0,12087
481577,15.516001,POINT (-81.41667 24.66667),2953.0,12087
481597,15.546000,POINT (-81.54167 24.625),2953.0,12087


In [38]:
filtered_points_with_counties.size

1884704

In [42]:
aggregated_df = filtered_points_with_counties.groupby('FIPS').agg(mean_temperature=('temperature', 'mean')).reset_index()


In [43]:
aggregated_df

,FIPS,mean_temperature
0,01001,6.052765
1,01003,6.715386
2,01005,5.527728
3,01007,6.123239
4,01009,5.990788
...,...,...
3097,56037,1.781057
3098,56039,-0.445345
3099,56041,0.589887
3100,56043,4.141521
